In [1]:
import sys, os
import numpy as np
import pandas as pd
import librosa

working_dir = os.getcwd()

while working_dir[-11:] != "Informatics":
    os.chdir("..")
    working_dir = os.getcwd()

In [2]:
path = "data/emotifymusic/"
genres = ["classical", "electronic", "pop", "rock"]

In [3]:
sys.path.append(os.getcwd())

import librosa.display
import matplotlib.pyplot as plt

def extractMFCCs(path, genre=None, print_message = True):
    """
    Extract MFCCs from sound files in folder
    """
    
    sample_rate = 44100
    
    n_fft = 8192
    win_length = int(0.025*sample_rate)
    hop_length = int(0.010*sample_rate)
    
    genre_shift = 0
    
    if genre=="electronic":
        genre_shift = 100
    elif genre=="pop":
        genre_shift = 200
    elif genre=="rock":
        genre_shift = 300
        
        
    INDEX = []
    MFCCs = []
    
    i = 0
    
    for file in os.scandir(path):
        if file.path.endswith(".mp3"):
            if print_message: print("analyzing ", file.name, file.path)

            currentpath = path + "/" + file.name
            int_track = int(file.name[:-4])
            int_track += genre_shift

            sound, sample_rate = librosa.load(currentpath, sr=None, mono=True) 
            mfcc = librosa.feature.mfcc(sound, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, win_length=win_length)

            if np.shape(mfcc)[1] < 6003:
                mfcc = np.append(mfcc, np.zeros((20,6003-np.shape(mfcc)[1])), axis=-1)

            elif np.shape(mfcc)[1] > 6003:
                mfcc = mfcc[:,6003]

            print(int_track, np.shape(mfcc))

            INDEX.append(int_track)
            MFCCs.append(mfcc)
        
    return MFCCs, INDEX
            # dataframe append file.name, sample_rate, duration, max_abs_amp, mean_abs_amp

In [4]:
genres = ["classical", "electronic", "pop", "rock"]

MFCC = []
INDEX = []

for genre in genres:
    mfccs, index = extractMFCCs(path+genre, genre=genre)
    mfccs = np.array(mfccs)
    index = np.array(index)
    
    index_ind = np.array(index).argsort()
    
    sorted_arr1 = index[index_ind[:]]
    sorted_arr2 = mfccs[index_ind[:]]
    
    INDEX.append(sorted_arr1)
    MFCC.append(sorted_arr2)
    
    

analyzing  1.mp3 data/emotifymusic/classical\1.mp3


C:\Users\sergi\Anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


1 (20, 6003)
analyzing  10.mp3 data/emotifymusic/classical\10.mp3
10 (20, 6003)
analyzing  100.mp3 data/emotifymusic/classical\100.mp3
100 (20, 6003)
analyzing  11.mp3 data/emotifymusic/classical\11.mp3
11 (20, 6003)
analyzing  12.mp3 data/emotifymusic/classical\12.mp3
12 (20, 6003)
analyzing  13.mp3 data/emotifymusic/classical\13.mp3
13 (20, 6003)
analyzing  14.mp3 data/emotifymusic/classical\14.mp3
14 (20, 6003)
analyzing  15.mp3 data/emotifymusic/classical\15.mp3
15 (20, 6003)
analyzing  16.mp3 data/emotifymusic/classical\16.mp3
16 (20, 6003)
analyzing  17.mp3 data/emotifymusic/classical\17.mp3
17 (20, 6003)
analyzing  18.mp3 data/emotifymusic/classical\18.mp3
18 (20, 6003)
analyzing  19.mp3 data/emotifymusic/classical\19.mp3
19 (20, 6003)
analyzing  2.mp3 data/emotifymusic/classical\2.mp3
2 (20, 6003)
analyzing  20.mp3 data/emotifymusic/classical\20.mp3
20 (20, 6003)
analyzing  21.mp3 data/emotifymusic/classical\21.mp3
21 (20, 6003)
analyzing  22.mp3 data/emotifymusic/classical\22.

C:\Users\sergi\AppData\Local\Temp/ipykernel_13912/3896068259.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mfccs = np.array(mfccs)


ValueError: could not broadcast input array from shape (20,6003) into shape (20,)

In [ ]:
index_ind = np.array(index).argsort()

index =  np.array(index)
mfccs = np.array(mfccs)

sorted_arr1 = index[index_ind[:]]
sorted_arr2 = mfccs[index_ind[:]]



In [ ]:
print(np.shape(sorted_arr2[15]))

fig, ax = plt.subplots(figsize=(15,5))
ax.imshow(sorted_arr2[15], interpolation='nearest', aspect='auto')
plt.show()

print(sorted_arr2[16][10])